In [1]:
from result_saver import SaverProvider
provider = SaverProvider()

In [2]:
import numpy as np
# parameter grid
distances = [30, 40, 55]
rounds = np.arange(5, 46, 2)
print(rounds)
devices = ["ibm_nazca", "ibm_sherbrooke", "ibm_brisbane", "ibm_cusco"]
logicals = ['0', '1']

[ 5  7  9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45]


# Sligthly faster faster triple loop

In [3]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import find_longest_path_in_hex
from Scratch import metadata_helper


hex_backend = provider.get_backend('ibm_sherbrooke')
path, _, _  = find_longest_path_in_hex(hex_backend)

for nb_of_rounds in tqdm(rounds):
    for distance in (distances):  
        bounded_path = path[:2 * distance - 1]
        layout = bounded_path[1::2] + bounded_path[::2] 
        code = RepetitionCodeCircuit(distance, nb_of_rounds) 
        for device in devices:
            backend = provider.get_backend(device)
            for logical in logicals:          
                qc = code.circuit[logical]                                   
                transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=3)

                n_shots = int(1e6 / distance**2)
                metadata = metadata_helper(descr="dist_55 INTERNAL ERROR TEST", code="RepetitionCodeCircuit", distance=distance, rounds=f"{nb_of_rounds}",  logical=logical, layout='_is_hex=True')
                backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single') # meas_level=1 for IQ  
                


  0%|          | 0/21 [00:00<?, ?it/s]

17:49:06 Warning: divide by zero encountered in det. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-Sk8aHGSa/lib/python3.11/site-packages/numpy/linalg/linalg.py, LINE: 2180
17:49:06 Warning: invalid value encountered in det. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-Sk8aHGSa/lib/python3.11/site-packages/numpy/linalg/linalg.py, LINE: 2180
17:49:07 Warning: updating metadata. IN FILE: /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46
17:49:21 Warning: The backend ibm_sherbrooke is currently paused.. IN FILE: /Users/mha/.local/share/virtualenvs/Soft-Info-Sk8aHGSa/lib/python3.11/site-packages/qiskit_ibm_provider/ibm_backend.py, LINE: 447
